# Combining Two or More DataFrames

In [1]:
# Loading libraries
import pandas as pd
import numpy as np
from os import path

# File path
DATA_DIR = '/Users/simmsjn/Documents/GitHub/ltcwff-files/data'

# loading dataframes
pg = pd.read_csv(path.join(DATA_DIR, 'player_game_2017_sample.csv'))  # player-game
games = pd.read_csv(path.join(DATA_DIR, 'game_2017_sample.csv'))  # game info
player = pd.read_csv(path.join(DATA_DIR, 'player_2017_sample.csv')) # player info


### Merging

    1. What columns are you joining on?
    2. Are you doing a 1:1, 1:many, many:1, or many:many type join?
    3. What are you doing with the unmatched obserevations?

#### Merge Q1: What columns are you joining on?

In [2]:
# Say we want to analyze whether QBs throw for more yards at home or away
# We need to merge two tables here since no table has both the player-game combos and the games being away or home
# We have two tables: games and pg

pd.merge(pg, games[['gameid', 'home', 'away']], on='gameid').head()

player_name  week  carries      gameid   player_id  rush_yards  \
0     T.Brady     1      0.0  2017090700  00-0019596         0.0   
1     A.Smith     1      2.0  2017090700  00-0023436         9.0   
2  D.Amendola     1      0.0  2017090700  00-0026035         0.0   
3  R.Burkhead     1      3.0  2017090700  00-0030288        15.0   
4     T.Kelce     1      1.0  2017090700  00-0030506         4.0   

   rush_fumbles  rush_tds  raw_yac  rec_fumbles  ...  pass_tds  air_tds  \
0           0.0       0.0      0.0          0.0  ...       0.0      0.0   
1           0.0       0.0      0.0          0.0  ...       4.0      1.0   
2           0.0       0.0     49.0          1.0  ...       0.0      0.0   
3           0.0       0.0      7.0          0.0  ...       0.0      0.0   
4           0.0       0.0     23.0          0.0  ...       0.0      0.0   

   season  team  pos  rec_yards  receptions  targets  home  away  
0    2017    NE   QB        0.0         0.0      0.0    NE    KC  
1    2017    KC   QB        0.0         0.0      0.0    NE    KC  
2    2017    NE   WR      100.0         6.0      7.0    NE    KC  
3    2017    NE   RB        8.0         1.0      3.0    NE    KC  
4    2017    KC   TE       40.0         5.0      7.0    NE    KC  

[5 rows x 31 columns]

#### Merging is precise

    To merge successfully, the vals in the columns you're linking must be exactly the same!

In [3]:
# Merging works on multiple columns as well

# creating two diff DFs both of which are at the player-game level

rushDf = pg[['gameid', 'player_id', 'rush_yards', 'rush_tds']]

recDf = pg[['gameid', 'player_id', 'rec_yards', 'rec_tds']]

In [4]:
# Combining them on player_id and gameid

combined = pd.merge(rushDf,recDf, on=['player_id','gameid'])
combined.head()

gameid   player_id  rush_yards  rush_tds  rec_yards  rec_tds
0  2017090700  00-0019596         0.0       0.0        0.0      0.0
1  2017090700  00-0023436         9.0       0.0        0.0      0.0
2  2017090700  00-0026035         0.0       0.0      100.0      0.0
3  2017090700  00-0030288        15.0       0.0        8.0      0.0
4  2017090700  00-0030506         4.0       0.0       40.0      0.0

#### Merge Q2: Are you doing a 1:1, 1:many, many:1, or many:many type join?

In [5]:
player.head()

player_id  season team pos player_name
0  00-0033951    2017  IND  RB      M.Mack
1  00-0028116    2017   SF  WR  A.Robinson
2  00-0033080    2017  CLE  TE   S.DeValve
3  00-0033553    2017  PIT  RB    J.Conner
4  00-0029615    2017  HOU  RB    L.Miller

Now we want to add back each player's name. However, we need to make sure of the merge type. 


In [6]:
# In the player table, there are no duplicates

player['player_id'].duplicated().any()

False

In [7]:
# How about the combined?
combined['player_id'].duplicated().any()

True

There are duplicates in the combined table! The player shows up once in each game played per week.

So, we can see that every one player_id in player table is being match to many player_id's in combined table

Hence, this is a one to many (1:m) type merge!

In [8]:
# And now the merge
pd.merge(combined,player).head()

gameid   player_id  rush_yards  rush_tds  rec_yards  rec_tds  season  \
0  2017090700  00-0019596         0.0       0.0        0.0      0.0    2017   
1  2017091705  00-0019596         9.0       0.0        0.0      0.0    2017   
2  2017092407  00-0019596         6.0       0.0        0.0      0.0    2017   
3  2017100107  00-0019596         2.0       0.0        0.0      0.0    2017   
4  2017100500  00-0019596         5.0       0.0        0.0      0.0    2017   

  team pos player_name  
0   NE  QB     T.Brady  
1   NE  QB     T.Brady  
2   NE  QB     T.Brady  
3   NE  QB     T.Brady  
4   NE  QB     T.Brady

In [9]:
# You can pass the type of merge you are expeccting using the validate keyword. Pandas will give error it not true

pd.merge(combined,player, validate = '1:1')

# MergeError: Merge keys are not unique in left dataset; not a one-to-one merge

MergeError: Merge keys are not unique in left dataset; not a one-to-one merge

#### Merge Q3: What are you doing with the unmatched observations?

In [12]:
rush_df = pg.loc[pg['rush_yards'] >0,
                ['gameid','player_id','rush_yards','rush_tds']]

In [14]:
rec_df = pg.loc[pg['rec_yards'] > 0,
               ['gameid','player_id', 'rec_yards', 'rec_tds']]

In [15]:
rush_df.shape

(555, 4)

In [16]:
rec_df.shape

(1168, 4)

In [17]:
# When merging these tables, Pandas defaults to keeping only the vals in BOTH tables
comb_inner = pd.merge(rush_df,rec_df)

In [19]:
comb_inner.shape

(355, 6)

In [22]:
#Alternatively we can keep everything in the left or right by passing 'left' or 'right' to the how arg

#left
comb_left = pd.merge(rush_df, rec_df, how='left')
comb_left.head()

gameid   player_id  rush_yards  rush_tds  rec_yards  rec_tds
0  2017090700  00-0023436         9.0       0.0        NaN      NaN
1  2017090700  00-0030288        15.0       0.0        8.0      0.0
2  2017090700  00-0030506         4.0       0.0       40.0      0.0
3  2017090700  00-0033923       148.0       1.0       98.0      2.0
4  2017091705  00-0019596         9.0       0.0        NaN      NaN

In [23]:
# right
comb_right = pd.merge(rush_df, rec_df, how='right')
comb_right.head()

gameid   player_id  rush_yards  rush_tds  rec_yards  rec_tds
0  2017090700  00-0026035         NaN       NaN      100.0      0.0
1  2017090700  00-0030288        15.0       0.0        8.0      0.0
2  2017090700  00-0030506         4.0       0.0       40.0      0.0
3  2017090700  00-0033923       148.0       1.0       98.0      2.0
4  2017091705  00-0030288         3.0       0.0       41.0      1.0

In [25]:
# an outer merge keeps everything. passing indicator=True gives a _merge column saying which table the row is from

comb_outer = pd.merge(rush_df, rec_df, how='outer', indicator = True)
comb_outer.head()
comb_outer.shape

(1368, 7)

In [26]:
# looking at value counts from the merge column

comb_outer['_merge'].value_counts()

right_only    813
both          355
left_only     200
Name: _merge, dtype: int64

#### More on pd.merge


In [2]:
rush_df.columns = ['gameid','player_id', 'rush_yards', 'rush-yards']
rec_df.columns = ['gameid','player_id', 'rec_yards', 'rec_tds']

NameError: name 'rush_df' is not defined